In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

base_dir = "/content/drive/My Drive"

Mounted at /content/drive


In [ ]:
import torch
import torch.nn as nn
import pickle
import os
from torch.utils.data import DataLoader,Dataset

import numpy as np

In [ ]:
class data(Dataset):

    def __init__(self, base_dir):
  
        self.base_dir = base_dir
        datapath = os.path.join(base_dir,'data_train.pkl')

        with open(datapath, 'rb') as f: 
            data_train = pickle.load(f)

        self.length = len(data_train['joint_3d'])

        self.pts2d = data_train['joint_2d_1']
        self.pts3d = data_train['joint_3d']

    def __len__(self):
        return self.length

    def __getitem__(self, idx):
        
        pts2di = self.pts2d[idx]
        pts3di = self.pts3d[idx]

        return pts2di,pts3di

In [ ]:
def cal_mpjpe(pose_1, pose_2, avg=True):
    n_joints = pose_1.shape[1]
    batch_size = pose_1.shape[0]
    pose_1 = pose_1.clone().reshape(batch_size, n_joints, 3)
    pose_2 = pose_2.clone().reshape(batch_size, n_joints, 3)
    diff = pose_1-pose_2
    diff_sq = diff ** 2
    dist_per_joint = torch.sqrt(torch.sum(diff_sq, axis=2))
    dist_per_sample = torch.mean(dist_per_joint, axis=1)
    if avg is True:
        dist_avg = torch.mean(dist_per_sample)
    else:
        dist_avg = dist_per_sample
    return dist_avg

In [ ]:
data_test = data(base_dir)

data_test = DataLoader(data_test,batch_size=64,shuffle=True, drop_last=False, pin_memory= True,num_workers=0)

In [ ]:
class LiftModel(nn.Module):

  def __init__(self,n_blocks=2, hidden_layer=1024, dropout=0.1, output_nodes=15*3):
    super(LiftModel, self).__init__()
    self.n_blocks = n_blocks
    self.hidden_layer = hidden_layer
    self.dropout = dropout
    self.output_nodes = output_nodes
    self.input_nodes = 15*2

    self.linear0 = nn.Linear(self.input_nodes, 1024)

    self.blocks = nn.ModuleList()

    self.block = nn.Sequential( 
        nn.Linear(1024,1024),
        nn.BatchNorm1d(1024),
        nn.ReLU(),
        nn.Dropout(self.dropout),
        nn.Linear(1024,1024),
        nn.BatchNorm1d(1024),
        nn.ReLU(),
        nn.Dropout(self.dropout)
        )
    
    for i in range(self.n_blocks):
      self.blocks.append(self.block)
  
    self.output = nn.Linear(1024,self.output_nodes)

  def forward(self,x):
      x = x.view(-1,30)
      x = self.linear0(x)
      for i in range(self.n_blocks):
        x = self.blocks[i](x)

      x = self.output(x)
      x = x.view(-1,15,3)
      
      return x

device = 'cuda' if torch.cuda.is_available() else 'cpu'   

model = LiftModel()
model = model.to(device)
model.load_state_dict(torch.load(os.path.join(base_dir,'bestModel.pt')),strict=False)

# if (torch.cuda.is_available()):
#   #checkpoint=torch.load(os.path.join(base_dir,'model_parameters'))
#   model.load_state_dict(torch.load(os.path.join(base_dir,'model_3')))
# else:
#   #checkpoint=torch.load(os.path.join(base_dir,'model_parameters'), map_location=torch.device('cpu'))
#   model.load_state_dict(torch.load(os.path.join(base_dir,'model_parameters_b'), map_location=torch.device('cpu')))


total_loss = 0
output = []
model.eval()
for input,target in data_test:
  input = input.to(device)
  target = target.to(device)
  predicted = model(input)
  loss = cal_mpjpe(predicted.to(device),target)

  total_loss+= loss.item()
  output.extend(predicted)

total_loss = total_loss / len(data_test)


In [ ]:
#print(total_loss,len(output))
print('Loss =',total_loss)

Loss = 1.147617555250887
